# Inception V3

In [1]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from pathlib import Path
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa

In [2]:
MODEL_ID = "iv3_sgd"

TRAIN_DIR = Path("../../data/train")
MODEL_DIR = Path("./models")
LOG_DIR = Path("./log")
CACHE_DIR = Path("./tmp/cache")

BATCH_SIZE = 64
VAL_SPLIT = 0.25
IMG_HEIGHT = IMG_WIDTH = 200
GPU_MEMORY_LIMIT = None
SEED = 1

NUM_EPOCHS = 100
INITIAL_EPOCH = 0
LOAD_MODEL_PATH = None

In [3]:
# Remove cache directory if available
if CACHE_DIR.exists():
    shutil.rmtree(CACHE_DIR, ignore_errors = True)

MODEL_DIR.mkdir(parents = True, exist_ok = True)
LOG_DIR.mkdir(parents = True, exist_ok = True)
CACHE_DIR.mkdir(parents = True, exist_ok = True)

In [4]:
# Set global seed for reproducible results
tf.random.set_seed(SEED)

In [5]:
# Limit GPU Memory Usage to GPU_MEMORY_LIMIT
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus and GPU_MEMORY_LIMIT:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit = GPU_MEMORY_LIMIT)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

## Loading Images

In [6]:
X_train, X_test = train_test_split(
    list(TRAIN_DIR.glob("**/*.jpg")),
    test_size = VAL_SPLIT,
    random_state = SEED
)

train_filepaths = []
train_classes = set()

val_filepaths = []
val_classes = set()

for filepath in X_train:
    train_filepaths.append(str(filepath))
    train_classes.add(filepath.parts[-2])
print("Train: Found {} images belonging to {} classes".format(len(train_filepaths), len(train_classes)))
    
for filepath in X_test:
    val_filepaths.append(str(filepath))
    val_classes.add(filepath.parts[-2])
print("Validation: Found {} images belonging to {} classes".format(len(val_filepaths), len(val_classes)))
    
train_filepaths = np.array(train_filepaths)
val_filepaths = np.array(val_filepaths)

classes = np.array(list(train_classes.union(val_classes)))
classes = np.sort(classes)

Train: Found 79044 images belonging to 42 classes
Validation: Found 26348 images belonging to 42 classes


In [7]:
train_fp_ds = tf.data.Dataset.from_tensor_slices(train_filepaths)
for f in train_fp_ds.take(3):
    print(f.numpy())

b'..\\..\\data\\train\\32\\5f2dba1fbf124f2db0f3918b37aeab9e.jpg'
b'..\\..\\data\\train\\19\\adf95fd3be74f7a53a7ea901d3043126.jpg'
b'..\\..\\data\\train\\01\\f7be16f17e1d3367f2d4d99a1c5544a6.jpg'


In [8]:
val_fp_ds = tf.data.Dataset.from_tensor_slices(val_filepaths)
for f in val_fp_ds.take(3):
    print(f.numpy())

b'..\\..\\data\\train\\04\\0272e1c91b21303d22915d5d2c384c7f.jpg'
b'..\\..\\data\\train\\25\\9e977c57d2686943ba637c92f51c9f7d.jpg'
b'..\\..\\data\\train\\21\\29443df248d863814e40ad72f0c9a65b.jpg'


In [9]:
def load_img(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    label = parts[-2] == classes
    
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels = 3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH]) 
    
    return img, label

In [10]:
train_img_ds = train_fp_ds.map(load_img, num_parallel_calls = tf.data.experimental.AUTOTUNE)
for image, label in train_img_ds.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

Image shape:  (200, 200, 3)
Label:  [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False  True False False False
 False False False False False False]


In [11]:
val_img_ds = val_fp_ds.map(load_img, num_parallel_calls = tf.data.experimental.AUTOTUNE)
for image, label in val_img_ds.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

Image shape:  (200, 200, 3)
Label:  [False False False False  True False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False]


## Image Preprocessing

In [12]:
def augment(img, label):
    def augment_inner(img, label):
        img = img.numpy()
        augmenters = iaa.OneOf([
            iaa.Affine(scale=(1.0, 1.4)),
            iaa.Affine(rotate=(-35, 35), mode="edge"),
            iaa.Affine(scale=(1.0, 1.2), rotate=(-22.5, 22.5), mode="edge"),
            iaa.LinearContrast((0.8, 1.3)),
            iaa.Sequential([iaa.Affine(scale=(1.0, 1.2)), iaa.LinearContrast((0.8, 1.3))]),
            iaa.Sequential([iaa.Affine(rotate=(-22.5, 22.5), mode="edge"), iaa.LinearContrast((0.7, 1.2))]),
            iaa.Fliplr(1),
            iaa.Sequential([iaa.Fliplr(1), iaa.Affine(rotate=(-35, 35), mode="edge")]),
            iaa.Sequential([iaa.Fliplr(1), iaa.Affine(scale=(1.0, 1.2))]),
            iaa.Sequential([iaa.Fliplr(1), iaa.Affine(scale=(1.0, 1.2), rotate=(-22.5, 22.5), mode="edge")]),
            iaa.GaussianBlur(sigma=(0.0, 1.3)),
            iaa.Cutout(nb_iterations=(1, 3), size=0.2)
        ]) 
        img = augmenters.augment_image(img)
        img = tf.clip_by_value(img, 0, 1)
        return img, label
    output = tf.py_function(augment_inner, [img, label], [tf.float32, tf.bool])
    output[0].set_shape(img.get_shape())
    output[1].set_shape(label.get_shape())
    return output

In [13]:
processed_train_ds = train_img_ds \
                        .cache(str(CACHE_DIR / "train.tfcache")) \
                        .map(augment, num_parallel_calls = tf.data.experimental.AUTOTUNE) \
                        .shuffle(BATCH_SIZE * 20, seed = SEED) \
                        .repeat() \
                        .batch(BATCH_SIZE) \
                        .prefetch(tf.data.experimental.AUTOTUNE)

In [14]:
processed_val_ds = val_img_ds \
                        .batch(BATCH_SIZE) \
                        .prefetch(tf.data.experimental.AUTOTUNE)

## Training Model

In [15]:
if not LOAD_MODEL_PATH:
    model = tf.keras.models.Sequential()
    model.add(tf.keras.applications.InceptionV3(
            include_top = False,
            pooling = "avg",
            input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)
    ))
    model.add(tf.keras.layers.Dense(2048, activation = "relu"))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(len(classes), activation = "softmax"))
    
    model.layers[0].trainable = False
    for layer in model.layers[0].layers:
        layer.trainable = False

    model.compile(
        optimizer = tf.keras.optimizers.SGD(),
        loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.1),
        metrics = ["accuracy"]
    )
else:
    model = tf.keras.models.load_model(str(LOAD_MODEL_PATH))

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 2048)              21802784  
_________________________________________________________________
dense (Dense)                (None, 2048)              4196352   
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 42)                86058     
Total params: 26,085,194
Trainable params: 4,282,410
Non-trainable params: 21,802,784
_________________________________________________________________


In [17]:
checkpoint_path = MODEL_DIR / (MODEL_ID + ".{epoch:03d}-{val_accuracy:.4f}.h5")
log_path = (LOG_DIR / MODEL_ID).with_suffix(".csv")

print(checkpoint_path)
print(log_path)

models\iv3_sgd.{epoch:03d}-{val_accuracy:.4f}.h5
log\iv3_sgd.csv


In [18]:
history = model.fit(
    processed_train_ds,
    epochs = NUM_EPOCHS,
    initial_epoch = INITIAL_EPOCH,
    steps_per_epoch = len(train_filepaths) // BATCH_SIZE,
    validation_data = processed_val_ds,
    validation_steps = len(val_filepaths) // BATCH_SIZE,
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            patience = 10,
            restore_best_weights = True,
            verbose = 1,
            monitor = "val_accuracy",
            mode = "max"
        ),
        tf.keras.callbacks.CSVLogger(str(log_path), append = True),
        tf.keras.callbacks.ModelCheckpoint(
            str(checkpoint_path),
            monitor = "val_accuracy",
            mode = "max",
            save_best_only = True
        )
    ]
)

Epoch 1/100
1235/1235 [==============================] - 529s 428ms/step - loss: 2.3374 - accuracy: 0.4728 - val_loss: 1.9614 - val_accuracy: 0.5875
Epoch 2/100
1235/1235 [==============================] - 403s 326ms/step - loss: 2.0464 - accuracy: 0.5579 - val_loss: 1.8968 - val_accuracy: 0.6124
Epoch 3/100
1235/1235 [==============================] - 406s 328ms/step - loss: 1.9771 - accuracy: 0.5828 - val_loss: 1.8638 - val_accuracy: 0.6225
Epoch 4/100
1235/1235 [==============================] - 416s 337ms/step - loss: 1.9360 - accuracy: 0.5957 - val_loss: 1.8345 - val_accuracy: 0.6330
Epoch 5/100
1235/1235 [==============================] - 417s 338ms/step - loss: 1.9018 - accuracy: 0.6072 - val_loss: 1.8171 - val_accuracy: 0.6401
Epoch 6/100
1235/1235 [==============================] - 424s 343ms/step - loss: 1.8742 - accuracy: 0.6164 - val_loss: 1.8085 - val_accuracy: 0.6401
Epoch 7/100
1235/1235 [==============================] - 421s 341ms/step - loss: 1.8533 - accuracy: 0.6209

1235/1235 [==============================] - 392s 317ms/step - loss: 1.4309 - accuracy: 0.7670 - val_loss: 1.6699 - val_accuracy: 0.6897
Epoch 55/100
1235/1235 [==============================] - 389s 315ms/step - loss: 1.4264 - accuracy: 0.7680 - val_loss: 1.6707 - val_accuracy: 0.6881
Epoch 56/100
1235/1235 [==============================] - 445s 360ms/step - loss: 1.4220 - accuracy: 0.7716 - val_loss: 1.6675 - val_accuracy: 0.6908
Epoch 57/100
1235/1235 [==============================] - 460s 373ms/step - loss: 1.4176 - accuracy: 0.7732 - val_loss: 1.6697 - val_accuracy: 0.6899
Epoch 58/100
1235/1235 [==============================] - 424s 344ms/step - loss: 1.4175 - accuracy: 0.7735 - val_loss: 1.6671 - val_accuracy: 0.6906
Epoch 59/100
1235/1235 [==============================] - 558s 452ms/step - loss: 1.4108 - accuracy: 0.7746 - val_loss: 1.6660 - val_accuracy: 0.6904
Epoch 60/100
1235/1235 [==============================] - 739s 598ms/step - loss: 1.4046 - accuracy: 0.7755 - val

KeyboardInterrupt: 

In [20]:
acc = history.history["accuracy"]
loss = history.history["loss"]
validation_acc = history.history["val_accuracy"]
validation_loss = history.history["val_loss"]

epochs_range = range(len(history.history["accuracy"]))

plt.figure(figsize = (8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label = "Training Accuracy")
plt.plot(epochs_range, validation_acc, label = "Validation Accuracy")
plt.legend(loc = "lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label = "Training Loss")
plt.plot(epochs_range, validation_loss, label = "Validation Loss")
plt.legend(loc = "lower right")
plt.title("Training and Validation Loss")
plt.show()

NameError: name 'history' is not defined

In [21]:
# Remove cache directory if available
if CACHE_DIR.exists():
    shutil.rmtree(CACHE_DIR, ignore_errors = True)